In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import torch
import librosa
import numpy as np
import soundfile as sf
from scipy.io import wavfile
from IPython.display import Audio
from transformers import Wav2Vec2ForCTC, Wav2Vec2Tokenizer
import string

In [ ]:
tokenizer = Wav2Vec2Tokenizer.from_pretrained("facebook/wav2vec2-base-960h")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")

/usr/local/lib/python3.8/dist-packages/transformers/models/wav2vec2/tokenization_wav2vec2.py:752: FutureWarning: The class `Wav2Vec2Tokenizer` is deprecated and will be removed in version 5 of Transformers. Please use `Wav2Vec2Processor` or `Wav2Vec2CTCTokenizer` instead.
  warnings.warn(


In [ ]:
file_name = 'sharvu_acc.wav'
Audio(file_name)

In [ ]:
ans = []

In [ ]:
data = wavfile.read(file_name)
framerate = data[0]
sounddata = data[1]
time = np.arange(0,len(sounddata))/framerate
print('Sampling rate:',framerate,'Hz')

Sampling rate: 44100 Hz


In [ ]:
input_audio, _ = librosa.load(file_name, sr=16000)


In [ ]:
input_values = tokenizer(input_audio, return_tensors="pt").input_values
logits = model(input_values).logits
predicted_ids = torch.argmax(logits, dim=-1)
transcription = tokenizer.batch_decode(predicted_ids)[0]  
print(transcription)
transcription = string.capwords(transcription)

HI MISSUS SHARVEY AN I ORDER IDY IS FIVE FOR SEVEN AND HAD ORDERED A PRODUCT WHICH IS A PAIR OF SHOES WHICH ISN'T THERE FOR DELIVERY YET THANK YOU


In [ ]:
!pip install stanza
import stanza

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# Setting path to install CoreNLP library to be able to use it 
corenlp_dir = './corenlp'
stanza.install_corenlp(dir=corenlp_dir)

import os
os.environ["CORENLP_HOME"] = corenlp_dir

!ls $CORENLP_HOME

build.xml				  LIBRARY-LICENSES
corenlp.sh				  LICENSE.txt
CoreNLP-to-HTML.xsl			  Makefile
ejml-core-0.39.jar			  patterns
ejml-core-0.39-sources.jar		  pom-java-11.xml
ejml-ddense-0.39.jar			  pom-java-17.xml
ejml-ddense-0.39-sources.jar		  pom.xml
ejml-simple-0.39.jar			  protobuf-java-3.19.2.jar
ejml-simple-0.39-sources.jar		  README.txt
input.txt				  RESOURCE-LICENSES
input.txt.out				  sample-project-pom.xml
input.txt.xml				  SemgrexDemo.java
istack-commons-runtime-3.0.7.jar	  ShiftReduceDemo.java
istack-commons-runtime-3.0.7-sources.jar  slf4j-api.jar
javax.activation-api-1.2.0.jar		  slf4j-simple.jar
javax.activation-api-1.2.0-sources.jar	  stanford-corenlp-4.5.1.jar
javax.json-api-1.0-sources.jar		  stanford-corenlp-4.5.1-javadoc.jar
javax.json.jar				  stanford-corenlp-4.5.1-models.jar
jaxb-api-2.4.0-b180830.0359.jar		  stanford-corenlp-4.5.1-sources.jar
jaxb-api-2.4.0-b180830.0359-sources.jar   StanfordCoreNlpDemo.java
jaxb-impl-2.4.0-b180830.0438.jar	  StanfordDependenci

In [ ]:
from stanza.server import CoreNLPClient

In [ ]:
client = CoreNLPClient(
    annotators=['tokenize','ssplit', 'pos', 'lemma', 'ner'], 
    memory='4G', 
    endpoint='http://localhost:9001',
    be_quiet=True)
print(client)
client.start()

import time; time.sleep(10)

INFO:stanza:Writing properties to tmp file: corenlp_server-db809ca39d5a4163.props
INFO:stanza:Starting server with command: java -Xmx4G -cp ./corenlp/* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9001 -timeout 60000 -threads 5 -maxCharLength 100000 -quiet True -serverProperties corenlp_server-db809ca39d5a4163.props -annotators tokenize,ssplit,pos,lemma,ner -preload -outputFormat serialized


In [ ]:
text2 = transcription
document = client.annotate(text2)

In [ ]:
num = ''
for sent in document.sentence:
    for m in sent.mentions:
      if m.entityType == 'NUMBER':
        num += m.entityMentionText+ ' '
        print("{:30s}\t{}".format(m.entityMentionText, m.entityType))
      # if m.entityType == 'PERSON':
      #   print("{:30s}\t{}".format(m.entityMentionText, 'NAMES'))
      if m.entityType == 'DATE' :
        print("{:30s}\t{}".format(m.entityMentionText, m.entityType))
        b = m.entityMentionText + '=>DATE'
        ans.append(b)
        
      # print(m)
    ans.append(num + ' Number')
ans

One                           	NUMBER
Three                         	NUMBER
Six                           	NUMBER
Tuesday                       	DATE


['Tuesday=>DATE', 'One Three Six  Number']

In [ ]:
client.stop()

In [ ]:
!pip install flair

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from flair.data import Sentence
from flair.models import SequenceTagger

In [ ]:
tagger = SequenceTagger.load("flair/ner-english-large")

/usr/local/lib/python3.8/dist-packages/huggingface_hub/file_download.py:594: FutureWarning: `cached_download` is the legacy way to download files from the HF hub, please consider upgrading to `hf_hub_download`
  warnings.warn(


2022-12-05 09:25:38,715 loading file /root/.flair/models/ner-english-large/07301f59bb8cb113803be316267f06ddf9243cdbba92a4c8067ef92442d2c574.554244d3476d97501a766a98078421817b14654496b86f2f7bd139dc502a4f29
2022-12-05 09:26:04,739 SequenceTagger predicts: Dictionary with 20 tags: <unk>, O, S-ORG, S-MISC, B-PER, E-PER, S-LOC, B-ORG, E-ORG, I-PER, S-PER, B-MISC, I-MISC, E-MISC, I-ORG, B-LOC, E-LOC, I-LOC, <START>, <STOP>


In [ ]:
sentence = Sentence(transcription)
tagger.predict(sentence)

for entity in sentence.get_spans('ner'):
  if entity.get_label("ner").value == 'PER':
    a = entity.text+ ' => PERSON'
    ans.append(a)
  if entity.get_label("ner").value == 'DATE':
    a = entity.text+ ' => DATE'
    ans.append(a)
  # print (entity)  



In [ ]:
dict = {"One":1, "Two":2, "Three": 3, "Four":4, "Five":5,"Six":6, "Seven":7, "Eight":8, "Nine":9}

def sad(ans):
  nos = ''
  for i in ans:
    if 'Number' in i:
      a = i.split()
      for j in range(len(a)-1):
        a[j] = dict[a[j]]
        nos += str(a[j])
      ans.remove(i)
      i = nos + ' => Orderid'
      ans.append(i)
      
  

sad(ans)
ans

['Tuesday=>DATE', 'Ruturage => PERSON', '136 => Orderid']

In [ ]:
ans

['Tuesday=>DATE', 'Ruturage => PERSON', '136 => Orderid']